# Dual Report: F1, Recall, Precision

# Pitch

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, precision_recall_fscore_support
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

# Load all CSV files
folder_path = r"G:\Shared drives\Prosody\RPT model\pitch"
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Combine data from all CSV files
data = []
annotations = []

for file in file_list:
    df = pd.read_csv(file, header=0)  # Assumes the first row is the header

    # The columns for the features in the CSVs are:
    # Min, Max, Mean, Standard Deviation, Z-Score, Duration, POS IDs
    features = df.iloc[:, [2, 3, 4, 5, 6, 9, 10, ]].values  # Select columns C, D, E, F, G, J

    # The columns for the labels are Prominence, and Boundary
    labels = df.iloc[:, [11, 12]].values  # Select columns K, L

    data.append(features)
    annotations.append(labels)

# Flatten the data into arrays
data = np.vstack(data)
annotations = np.vstack(annotations)

# Normalize the feature data
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(data, annotations, test_size=0.2, random_state=42)

# Reshape data for LSTM
time_steps = 1  # Adjust if needed for sequences
X_train = X_train.reshape((X_train.shape[0], time_steps, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], time_steps, X_test.shape[1]))

# Define the LSTM model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(time_steps, X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(annotations.shape[1], activation='sigmoid')  # Use sigmoid for multi-label binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Define the model save path
model_save_path = os.path.join(folder_path, "Pitch_LSTM_model.h5")

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Save the model
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Make predictions
#y_pred = (model.predict(X_test) > 0.4).astype(int)

# Calculate precision, recall, F1-score
#print("\nClassification Report:")
#print(classification_report(y_test, y_pred, target_names=[f"Class {i}" for i in range(annotations.shape[1])]))

# Compute and display combined scores
#precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='micro')
#print(f"Combined Precision: {precision:.4f}, Combined Recall: {recall:.4f}, Combined F1-Score: {f1:.4f}")
# Make predictions
y_pred_raw = model.predict(X_test)

# Apply different thresholds
y_pred = np.zeros_like(y_pred_raw)
y_pred[:, 0] = (y_pred_raw[:, 0] > 0.4).astype(int)  # Prominence threshold
y_pred[:, 1] = (y_pred_raw[:, 1] > 0.16).astype(int)  # Boundary threshold

# Calculate precision, recall, F1-score
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Prominence", "Boundary"]))

# Compute and display combined scores
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='micro')
print(f"Combined Precision: {precision:.4f}, Combined Recall: {recall:.4f}, Combined F1-Score: {f1:.4f}")

Epoch 1/20


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


726/726 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9986 - loss: 0.3071 - val_accuracy: 0.9986 - val_loss: 0.1648
Epoch 2/20
726/726 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9985 - loss: 0.1411 - val_accuracy: 0.9986 - val_loss: 0.1334
Epoch 3/20
726/726 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9981 - loss: 0.1272 - val_accuracy: 0.9981 - val_loss: 0.1253
Epoch 4/20
726/726 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9980 - loss: 0.1201 - val_accuracy: 0.9967 - val_loss: 0.1202
Epoch 5/20
726/726 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9972 - loss: 0.1151 - val_accuracy: 0.9952 - val_loss: 0.1180
Epoch 6/20
726/726 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9970 - loss: 0.1134 - val_accuracy: 0.9950 - val_loss: 0.1190
Epoch 7/20
726/726 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9969 - loss: 0.1113 - val_accuracy: 0.9960 - val_loss: 0.1130
Epoch 8/20
726/726 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9969 - loss: 0.1098 - val_accuracy: 0.9960 - val_

Model saved to: G:\My Drive\LABS\Experimental Linguistics lab\retraining csvs\pitch\Pitch_LSTM_model.h5
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - accuracy: 0.9964 - loss: 0.1059
Test Loss: 0.10589616745710373, Test Accuracy: 0.996417224407196
227/227 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Classification Report:
              precision    recall  f1-score   support

  Prominence       0.91      0.96      0.93      4393
    Boundary       0.51      0.86      0.64       190

   micro avg       0.89      0.95      0.92      4583
   macro avg       0.71      0.91      0.79      4583
weighted avg       0.90      0.95      0.92      4583
 samples avg       0.57      0.58      0.57      4583

Combined Precision: 0.8865, Combined Recall: 0.9511, Combined F1-Score: 0.9177


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

# Intensity

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, precision_recall_fscore_support
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

# Load all CSV files
folder_path = r"G:\Shared drives\Prosody\RPT model\intensity"
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Combine data from all CSV files
data = []
annotations = []

for file in file_list:

    # The columns for the features in the CSVs are:
    # Min, Max, Mean, Standard Deviation, Z-Score, Duration, POS IDs
    df = pd.read_csv(file, header=0)  # Assumes the first row is the header
    features = df.iloc[:, [2, 3, 4, 5, 6, 9, 10, ]].values  # Select columns C, D, E, F, G, J

    # The columns for the labels are Prominence, and Boundary
    labels = df.iloc[:, [11, 12]].values  # Select columns K, L

    data.append(features)
    annotations.append(labels)

# Flatten the data into arrays
data = np.vstack(data)
annotations = np.vstack(annotations)

# Normalize the feature data
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(data, annotations, test_size=0.2, random_state=42)

# Reshape data for LSTM
time_steps = 1  # Adjust if needed for sequences
X_train = X_train.reshape((X_train.shape[0], time_steps, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], time_steps, X_test.shape[1]))

# Define the LSTM model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(time_steps, X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(annotations.shape[1], activation='sigmoid')  # Use sigmoid for multi-label binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Define the model save path
model_save_path = os.path.join(folder_path, "Intensity_LSTM_model.h5")

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Save the model
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


# Make predictions
y_pred_raw = model.predict(X_test)

# Apply different thresholds
y_pred = np.zeros_like(y_pred_raw)
y_pred[:, 0] = (y_pred_raw[:, 0] > 0.4).astype(int)  # Prominence threshold
y_pred[:, 1] = (y_pred_raw[:, 1] > 0.16).astype(int)  # Boundary threshold

# Calculate precision, recall, F1-score
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Prominence", "Boundary"]))

# Compute and display combined scores
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='micro')
print(f"Combined Precision: {precision:.4f}, Combined Recall: {recall:.4f}, Combined F1-Score: {f1:.4f}")

Epoch 1/20


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


683/683 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9949 - loss: 0.3182 - val_accuracy: 0.9883 - val_loss: 0.1894
Epoch 2/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9600 - loss: 0.1702 - val_accuracy: 0.9269 - val_loss: 0.1641
Epoch 3/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9175 - loss: 0.1590 - val_accuracy: 0.9082 - val_loss: 0.1576
Epoch 4/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9014 - loss: 0.1537 - val_accuracy: 0.9187 - val_loss: 0.1528
Epoch 5/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9026 - loss: 0.1501 - val_accuracy: 0.9035 - val_loss: 0.1489
Epoch 6/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8956 - loss: 0.1462 - val_accuracy: 0.9260 - val_loss: 0.1580
Epoch 7/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8997 - loss: 0.1441 - val_accuracy: 0.8890 - val_loss: 0.1616
Epoch 8/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9033 - loss: 0.1414 - val_accuracy: 0.9086 - val_

Model saved to: G:\My Drive\LABS\Experimental Linguistics lab\retraining csvs\intensity\Intensity_LSTM_model.h5
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9451 - loss: 0.1253
Test Loss: 0.12533189356327057, Test Accuracy: 0.9450549483299255
214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Classification Report:
              precision    recall  f1-score   support

  Prominence       0.95      0.95      0.95      3585
    Boundary       0.58      0.79      0.67       558

   micro avg       0.88      0.93      0.90      4143
   macro avg       0.76      0.87      0.81      4143
weighted avg       0.90      0.93      0.91      4143
 samples avg       0.48      0.50      0.48      4143

Combined Precision: 0.8827, Combined Recall: 0.9281, Combined F1-Score: 0.9048


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average

# 2CSV Test

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, precision_recall_fscore_support
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from scipy.interpolate import interp1d

# Load and process intensity data
def load_data(folder_path):
    file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
    data = []
    annotations = []
    for file in file_list:
        df = pd.read_csv(file, header=0)
        features = df.iloc[:, [2, 3, 4, 5, 6, 9, 10]].values  # Select columns C, D, E, F, G, J
        labels = df.iloc[:, [11, 12]].values  # Select columns K, L
        data.append(features)
        annotations.append(labels)
    data = np.vstack(data)
    annotations = np.vstack(annotations)
    return data, annotations

# Define the LSTM model
def create_lstm_model(input_shape, output_shape):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(output_shape, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Load intensity and pitch data
folder_path_intensity = r"G:\Shared drives\Prosody\RPT model\intensity"
folder_path_pitch = r"G:\Shared drives\Prosody\RPT model\pitch"

intensity_data, intensity_annotations = load_data(folder_path_intensity)
pitch_data, pitch_annotations = load_data(folder_path_pitch)

# Normalize the data
scaler = MinMaxScaler()
intensity_data = scaler.fit_transform(intensity_data)
pitch_data = scaler.transform(pitch_data)  # Use the same scaler for both

# Split data into training and testing
X_train_intensity, X_test_intensity, y_train_intensity, y_test_intensity = train_test_split(
    intensity_data, intensity_annotations, test_size=0.2, random_state=42
)
X_train_pitch, X_test_pitch, y_train_pitch, y_test_pitch = train_test_split(
    pitch_data, pitch_annotations, test_size=0.2, random_state=42
)

# Reshape data for LSTM
time_steps = 1
X_train_intensity = X_train_intensity.reshape((X_train_intensity.shape[0], time_steps, X_train_intensity.shape[1]))
X_test_intensity = X_test_intensity.reshape((X_test_intensity.shape[0], time_steps, X_test_intensity.shape[1]))
X_train_pitch = X_train_pitch.reshape((X_train_pitch.shape[0], time_steps, X_train_pitch.shape[1]))
X_test_pitch = X_test_pitch.reshape((X_test_pitch.shape[0], time_steps, X_test_pitch.shape[1]))

# Create models
intensity_model = create_lstm_model((time_steps, X_train_intensity.shape[2]), y_train_intensity.shape[1])
pitch_model = create_lstm_model((time_steps, X_train_pitch.shape[2]), y_train_pitch.shape[1])

# Train the models
intensity_model.fit(X_train_intensity, y_train_intensity, epochs=20, batch_size=32, validation_split=0.2)
pitch_model.fit(X_train_pitch, y_train_pitch, epochs=20, batch_size=32, validation_split=0.2)

# Make predictions (raw output/logits)
y_pred_intensity_raw = intensity_model.predict(X_test_intensity)
y_pred_pitch_raw = pitch_model.predict(X_test_pitch)

# Interpolation to match the lengths of both prediction arrays
def interpolate_predictions(predictions, target_length):
    # Create a linear interpolation function
    x = np.linspace(0, len(predictions) - 1, len(predictions))
    f = interp1d(x, predictions, axis=0, fill_value="extrapolate")
    x_new = np.linspace(0, len(predictions) - 1, target_length)
    return f(x_new)

# Determine the max length between the two sets of predictions
max_length = max(len(y_pred_intensity_raw), len(y_pred_pitch_raw))

# Interpolate predictions from both models to the same length
y_pred_intensity_interpolated = interpolate_predictions(y_pred_intensity_raw, max_length)
y_pred_pitch_interpolated = interpolate_predictions(y_pred_pitch_raw, max_length)

# Step 1: Combine the raw predictions using simple averaging (you could apply weighted averaging if needed)
y_pred_combined_raw = (y_pred_intensity_interpolated + y_pred_pitch_interpolated) / 2

# Step 2: Apply sigmoid activation to the combined raw predictions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

y_pred_combined_prob = sigmoid(y_pred_combined_raw)

# Step 3: Apply thresholds for each class
y_pred_combined_final = np.zeros_like(y_pred_combined_prob)

# Apply different thresholds for each class
# Prominence (class 0) with a higher threshold (e.g., 0.6)
y_pred_combined_final[:, 0] = (y_pred_combined_prob[:, 0] > 0.5).astype(int)

# Boundary (class 1) with a lower threshold (e.g., 0.4)
y_pred_combined_final[:, 1] = (y_pred_combined_prob[:, 1] > 0.4).astype(int)

# Ensure the length of predictions matches the length of ground truth
min_length = min(len(y_test_intensity), len(y_pred_combined_final))
y_test_intensity = y_test_intensity[:min_length]
y_pred_combined_final = y_pred_combined_final[:min_length]

# Now calculate the classification report
print("\nClassification Report:")
print(classification_report(y_test_intensity, y_pred_combined_final, target_names=["Prominence", "Boundary"]))

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_test_intensity, y_pred_combined_final, average='micro')
print(f"Combined Precision: {precision:.4f}, Combined Recall: {recall:.4f}, Combined F1-Score: {f1:.4f}")

combined_folder_path = r"G:\Shared drives\Prosody\RPT model" # change as needed
model.save(os.path.join(combined_folder_path, "Combined_LSTM_model.h5"))

Epoch 1/20


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


683/683 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9908 - loss: 0.3315 - val_accuracy: 0.9733 - val_loss: 0.1919
Epoch 2/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9422 - loss: 0.1695 - val_accuracy: 0.8912 - val_loss: 0.1696
Epoch 3/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8931 - loss: 0.1524 - val_accuracy: 0.9048 - val_loss: 0.1514
Epoch 4/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8867 - loss: 0.1452 - val_accuracy: 0.8930 - val_loss: 0.1447
Epoch 5/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8792 - loss: 0.1412 - val_accuracy: 0.8711 - val_loss: 0.1444
Epoch 6/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8758 - loss: 0.1387 - val_accuracy: 0.8877 - val_loss: 0.1474
Epoch 7/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8741 - loss: 0.1368 - val_accuracy: 0.8588 - val_loss: 0.1404
Epoch 8/20
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8745 - loss: 0.1362 - val_accuracy: 0.8778 - val_

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



Classification Report:
              precision    recall  f1-score   support

  Prominence       0.53      1.00      0.69      3585
    Boundary       0.08      1.00      0.15       558

   micro avg       0.30      1.00      0.47      4143
   macro avg       0.30      1.00      0.42      4143
weighted avg       0.47      1.00      0.62      4143
 samples avg       0.30      0.53      0.38      4143

Combined Precision: 0.3035, Combined Recall: 1.0000, Combined F1-Score: 0.4657
